In [20]:
import pandas as pd
from utils import *
import os
import fitz
from pdf2image import convert_from_path
import pytesseract
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

In [15]:
os.chdir('/Users/judyhe/Desktop/ten strands/ten strands research/policy-language-analysis/data/BP_7111')

In [30]:
policy_pdf_collection_doc_path = "BP_7111.xlsx" # insert pdf collection doc path here

policy_pdfs = pd.read_excel(policy_pdf_collection_doc_path)

In [31]:
policy_pdfs = extract_and_update_pdf_texts(policy_pdfs, 'BP7111: Path to PDF', 'BP7111: PDF Text')

# check all relevant columns and pdfs have been extracted
check_and_report_missing_texts(policy_pdfs, 'BP: 7111 Evaluating Existing Buildings', 'BP7111: Path to PDF', 'BP7111: PDF Text')

/Users/judyhe/Desktop/ten strands/ten strands research/policy-language-analysis/extraction_and_cleaning_notebooks/utils.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Book
Board Policies
Section
7000 - Facilities
Title
Evaluating Existing Buildings
Code
7111
Status
Active
Adopted
August 23, 2001
Last Revised
June 28, 2012
EVALUATING EXISTING BUILDINGS
 
 
The adequacy and design of all existing school facilities shall be evaluated periodically in terms of meeting the
needs of the instructional program and fulfilling the requirements of law regarding safety and structural
soundness, accessibility to the individuals with disabilities and energy conservation.
 
A primary consideration in evaluating existing facilities shall be the preservation of a healthful environment for
students and staff as well as providing an aesthetically pleasing learning and working atmosphere.
 
In the evaluation of existin

All relevant rows have text extracted.


In [32]:
def to_lowercase(text):
    '''Lower case'''
    return text.lower()

def remove_punctuation(text):
    '''Remove punctuation'''
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    '''Remove stopwords'''
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def remove_space(text):
    '''Remove multiple spaces with a single space'''
    return re.sub(r'\s+', ' ', text)

def remove_newlines(text):
    '''Remove newlines with a space'''
    return text.replace('\n', ' ')

def remove_leading_trailing_space(text):
    '''Remove leading/trailing space'''
    return text.strip()

def tokenize(text):
    '''Tokenize text'''
    return word_tokenize(text)

def remove_cf_pattern(text):
    '''Remove cf pattern in text'''
    pattern = r'\(\s*cf[^()]*\)'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

def remove_after_references(text):
    '''Remove legal references section if present'''
    if isinstance(text, str):
        references = ['legal reference', 'policy reference', 'state reference']
        for ref in references:
            if ref in text:
                return text.split(ref, 1)[0].strip()
    return text

In [33]:
def apply_transformations(text):
    text = remove_newlines(text)
    # text = to_lowercase(text)
    # text = remove_stopwords(text)
    text = remove_cf_pattern(text)
    # text = remove_punctuation(text)
    text = remove_after_references(text)
    return text

# Apply transformations to the 'PDF Text' column conditionally
policy_pdfs.loc[policy_pdfs['BP: 7111 Evaluating Existing Buildings'] == 1, 'BP7111: PDF Text'] = \
    policy_pdfs.loc[policy_pdfs['BP: 7111 Evaluating Existing Buildings'] == 1, 'BP7111: PDF Text'].apply(apply_transformations)

In [45]:
policy_pdfs = policy_pdfs.iloc[:,:-2]

In [46]:
policy_pdfs.to_csv('../../cleaned_data/BP_7111_cleaned.csv', index='False')